# TP 4 : Introduction au deep learning pour le NLP


Dans ce TP; on changera un peu des bases classiques d'apprentissage statistique, pour traiter un type de données différentes : le texte. 

Pour des raisons de temps, et comme il n'y a qu'une seule séance prévue sur ce sujet, on se focalisera sur les outils deep learning, et l'on ne verra pas les algorithmes type LDA/etc. 


## Données et preprocessing

En texte, les données sont rarement aussi propres que dans les bases utilisées jusqu'à maintenant. 
Pour ce TP, on téléchargera une base de données de 50 000 descriptions de films. 

### 1. Télécharger les données (https://www.kaggle.com/rounakbanik/the-movies-dataset ), et les ouvrir avec pandas.

### 2. Observer les résumés de films. Que remarquez-vous? Quelles sont vos idées de preprocessing? 

### 3. Quelles sont les variables intéressantes à prédire? Tracer leurs histogrammes. Que remarquez-vous? 

## Un premier modèle de ML classique

### 4. Faire un premier modèle de ML classique avec scikit learn en utilisant les variables non textuelles, évaluer.


## Embeddings

### 5. Apprendre des embeddings sur le corpus des descriptions des films non traité. Que remarquez-vous? 

### 6. Mettre en place le preprocessing adapté pour réduire la taille du vocabulaire et générer des embeddings de bonne qualité

## Des embeddings au modèle de prédiction

### 7. Quel problème remarquez-vous pour passer des embeddings à un modèle de prédiction? Suggérez des solutions d'aggrégation. Intégrez les résultats au modèle et évaluer. 

### 8. (Bonus) Implémenter un modèle de prédiction avec un LSTM simple. 

